**California Housing Prices**

In [ ]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2 --tf_xla_enable_xla_devices=false"
# os.environ["XLA_FLAGS"] = "--xla_gpu_autotune_level=0"

import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns 
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer,Dense, Normalization
from tensorflow.keras.metrics import RootMeanSquaredError


**Loading and Visualization Dataset**

In [ ]:
data=pd.read_csv("housing.csv")
data.head()

In [ ]:
data_header=data.columns
#sns.pairplot(data[data_header],diag_kind="kde")

**Data preparation**

*Data Cleaning*

In [ ]:
'''Checking is there any NaN value or not'''
data.isna().sum()

In [ ]:
data.mean(numeric_only=True)

In [ ]:
'''Filling nan value with mean'''
data=data.fillna(data.mean(numeric_only=True))
data.isna().sum().sum()

In [ ]:
data.dtypes

In [ ]:
tensor_data=tf.random.shuffle(tf.constant(data.select_dtypes(["float64"])))
tensor_data[:5]

In [ ]:
x=tensor_data[:,:-1]
x.shape

In [ ]:
y=tf.expand_dims(tensor_data[:,-1],1)
y.shape

*Data Spliting*

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
DATASET_SIZE=len(x)

In [ ]:
x_train=x[:int(DATASET_SIZE*TRAIN_RATIO)]
y_train=y[:int(DATASET_SIZE*TRAIN_RATIO)]

In [ ]:
x_val=x[int(DATASET_SIZE*TRAIN_RATIO):int(DATASET_SIZE*(TRAIN_RATIO+VAL_RATIO))]
y_val=y[int(DATASET_SIZE*TRAIN_RATIO):int(DATASET_SIZE*(TRAIN_RATIO+VAL_RATIO))]

In [ ]:
x_test=x[int(DATASET_SIZE*(TRAIN_RATIO+VAL_RATIO)):]
y_test=y[int(DATASET_SIZE*(TRAIN_RATIO+VAL_RATIO)):]

In [ ]:
normalizer=Normalization()
normalizer.adapt(x_train)
#normalizer(x_train)[:5]

**Model Preparation**

In [ ]:
model=Sequential([InputLayer(shape=(x_train.shape[1],)),
                  normalizer,
                  Dense(128,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(1)
                  ])
# model.summary()
# tf.keras.utils.plot_model(model,"model.png",True)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.2),
              loss=MeanSquaredError(),
              metrics=[RootMeanSquaredError])

**Model Training**

In [ ]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=100,verbose=1)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(['train loss','val_loss'])
plt.show()

In [ ]:
plt.plot(history.history["root_mean_squared_error"])
plt.plot(history.history["val_root_mean_squared_error"])
plt.title("model performance")
plt.xlabel("epoch")
plt.ylabel("rmse")
plt.legend(['train error',"val error"])
plt.show()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_true = list(y_test[:,0].numpy())
y_pred = list(model.predict(x_test)[:,0])
print(y_true)
print(y_pred)

In [ ]:
ind = np.arange(len(x_test))
plt.figure(figsize=(20,5))

width = 0.4

plt.bar(ind, y_pred, width, label='Predicted House Price')
plt.bar(ind + width, y_true, width, label='Actual House Price')

plt.xlabel('Actual vs Predicted Prices')
plt.ylabel('House Prices')

plt.show()